In [1]:
import pandas as pd
import numpy as np


In [2]:

df=pd.read_excel('prere_esc_raw_data.xlsx')

df=df.copy()

In [3]:
df.head()

,BUCKETNAME,DISTRIBUTABLE,RATING,COUPON,COUPONTYPE,AMT,USEOFPROCEEDS,TAXSTATUS,AVERAGELIFEDATE,MATURITY,CALLPRICE,REDEMPTIONPRICE,REDEMPTION_AMT,REDEMPTION_DATE,CALLDATE,REDEMPTION_TYPE,STATUSTYPE,STATUSSUBTYPE
0,NEW ISSUANCE,N,Escrowed,5.0,Fixed: Plain Vanilla Fixed Coupon,N,Hospitals,EXMP,NaN,11/15/2024,NaN,100,6510000,11/15/2024,NaN,Escrowed to Maturity,Active,Escrowed to Maturity
1,NEW ISSUANCE,N,Escrowed,5.0,Fixed: Plain Vanilla Fixed Coupon,N,Hospitals,EXMP,NaN,11/15/2023,NaN,100,6200000,11/15/2023,NaN,Escrowed to Maturity,Active,Escrowed to Maturity
2,C3,Y,Pre refunded,5.0,Fixed: Plain Vanilla Fixed Coupon,NaN,Primary & Secondary Education,EXMP,NaN,5/15/2026,100.0,100,3925000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called
3,C3,Y,Pre refunded,5.0,Fixed: Plain Vanilla Fixed Coupon,NaN,Primary & Secondary Education,EXMP,NaN,5/15/2025,100.0,100,3760000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called
4,C3,Y,Pre refunded,5.0,Fixed: Plain Vanilla Fixed Coupon,NaN,Primary & Secondary Education,EXMP,NaN,5/15/2024,100.0,100,3595000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BUCKETNAME       32 non-null     object 
 1   DISTRIBUTABLE    32 non-null     object 
 2   RATING           32 non-null     object 
 3   COUPON           32 non-null     float64
 4   COUPONTYPE       32 non-null     object 
 5   AMT              23 non-null     object 
 6   USEOFPROCEEDS    32 non-null     object 
 7   TAXSTATUS        32 non-null     object 
 8   AVERAGELIFEDATE  0 non-null      float64
 9   MATURITY         32 non-null     object 
 10  CALLPRICE        28 non-null     float64
 11  REDEMPTIONPRICE  32 non-null     int64  
 12  REDEMPTION_AMT   32 non-null     int64  
 13  REDEMPTION_DATE  32 non-null     object 
 14  CALLDATE         28 non-null     object 
 15  REDEMPTION_TYPE  32 non-null     object 
 16  STATUSTYPE       32 non-null     object 
 17  STATUSSUBTYPE    3

In [7]:

df['MATURITY'] = pd.to_datetime(df['MATURITY'])


In [8]:
df['Analysis']= 0

In [9]:
def prere_esc(df):
    #Pre refunded bonds spread

    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Pre refunded') & (df['COUPON'] == 0),['Analysis']] = 'new pref, spread 20.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Pre refunded') & (df['TAXSTATUS'] == 'AMT'),['Analysis']] = 'new pref, spread 20.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Pre refunded') & (df['COUPON'] == 0) & (df['TAXSTATUS'] == 'AMT'),['Analysis']] = 'new pref, spread 40.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Pre refunded') & (df['USEOFPROCEEDS'].isin(['Hospitals','Healthcare'])),['Analysis']] = 'new pref, spread 20.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Pre refunded') & (df['USEOFPROCEEDS'].isin(['Retirement Centers','Charter School'])),['Analysis']] = 'new pref, spread 30.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Pre refunded') & (df['COUPON'] != 0) & (df['TAXSTATUS'] != 'AMT') & (df['USEOFPROCEEDS'].isin(['Hospitals','Healthcare','Retirement Centers','Charter School']) == False),['Analysis']] = 'new pref, spread 0.'

    #Escrowed bonds spread

    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Escrowed') & (df['COUPON'] == 0) & (df['MATURITY'].between('01/01/2022','12/31/2022')),['Analysis']] = 'new etm, spread 15.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Escrowed') & (df['COUPON'] == 0) & (df['MATURITY'] >= '01/01/2023'),['Analysis']] = 'new etm, spread 20.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Escrowed') & (df['USEOFPROCEEDS'].isin(['Hospitals','Healthcare'])),['Analysis']] = 'new etm, spread 20.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Escrowed') & (df['USEOFPROCEEDS'].isin(['Retirement Centers','Charter School'])),['Analysis']] = 'new etm, spread 30.'
    df.loc[(df['Analysis'] == 0) & (df['RATING'] == 'Escrowed') & (df['COUPON'] != 0) & (df['TAXSTATUS'] != 'AMT') & (df['USEOFPROCEEDS'].isin(['Hospitals','Healthcare','Retirement Centers','Charter School']) == False),['Analysis']] = 'new etm, spread 0.'
    
    return df

prere_esc(df)



,BUCKETNAME,DISTRIBUTABLE,RATING,COUPON,COUPONTYPE,AMT,USEOFPROCEEDS,TAXSTATUS,AVERAGELIFEDATE,MATURITY,CALLPRICE,REDEMPTIONPRICE,REDEMPTION_AMT,REDEMPTION_DATE,CALLDATE,REDEMPTION_TYPE,STATUSTYPE,STATUSSUBTYPE,Analysis
0,NEW ISSUANCE,N,Escrowed,5.000,Fixed: Plain Vanilla Fixed Coupon,N,Hospitals,EXMP,NaT,2024-11-15,NaN,100,6510000,11/15/2024,NaN,Escrowed to Maturity,Active,Escrowed to Maturity,"new etm, spread 20."
1,NEW ISSUANCE,N,Escrowed,5.000,Fixed: Plain Vanilla Fixed Coupon,N,Hospitals,EXMP,NaT,2023-11-15,NaN,100,6200000,11/15/2023,NaN,Escrowed to Maturity,Active,Escrowed to Maturity,"new etm, spread 20."
2,C3,Y,Pre refunded,5.000,Fixed: Plain Vanilla Fixed Coupon,NaN,Primary & Secondary Education,EXMP,NaT,2026-05-15,100.0,100,3925000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
3,C3,Y,Pre refunded,5.000,Fixed: Plain Vanilla Fixed Coupon,NaN,Primary & Secondary Education,EXMP,NaT,2025-05-15,100.0,100,3760000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
4,C3,Y,Pre refunded,5.000,Fixed: Plain Vanilla Fixed Coupon,NaN,Primary & Secondary Education,EXMP,NaT,2024-05-15,100.0,100,3595000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
5,C8,Y,Pre refunded,5.000,Fixed: Plain Vanilla Fixed Coupon,NaN,Education,EXMP,NaT,2043-01-03,100.0,100,75000,2028-01-03 00:00:00,2028-01-03 00:00:00,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
6,GO - NONE - AA+ - CPN B/W 4% & 4.99%,Y,Pre refunded,4.000,Fixed: Plain Vanilla Fixed Coupon,N,Primary & Secondary Education,EXMP,NaT,2030-05-15,100.0,100,4610000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
7,GO - NONE - AA+ - CPN B/W 4% & 4.99%,Y,Pre refunded,4.000,Fixed: Plain Vanilla Fixed Coupon,N,Primary & Secondary Education,EXMP,NaT,2029-05-15,100.0,100,4445000,5/15/2023,5/15/2023,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
8,GO BQ - AA,Y,Pre refunded,3.250,Fixed: Plain Vanilla Fixed Coupon,N,Primary & Secondary Education,EXMP,NaT,2027-01-06,100.0,100,170000,2024-01-06 00:00:00,2023-06-05 00:00:00,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
9,GO BQ - AA-,Y,Pre refunded,3.375,Fixed: Plain Vanilla Fixed Coupon,N,Primary & Secondary Education,EXMP,NaT,2039-01-03,100.0,100,2270000,2027-01-03 00:00:00,2027-01-03 00:00:00,Pre-Refunded,Active,To Be Called,"new pref, spread 0."
